In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report,balanced_accuracy_score

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
file_paths = ["/kaggle/input/dswefw/final_data1.parquet" , "/kaggle/input/dswefw/final_data2.parquet" ,"/kaggle/input/dswefw/final_data3.parquet" ,
             "/kaggle/input/dswefw/final_data4.parquet" ,"/kaggle/input/dswefw/final_data5.parquet" ,"/kaggle/input/dswefw/final_data6.parquet" ,
             "/kaggle/input/dswefw/final_data7.parquet" ,"/kaggle/input/dswefw/final_data8.parquet" ,"/kaggle/input/dswefw/final_data9.parquet" ,
             "/kaggle/input/dswefw/final_data10.parquet"]


In [3]:
!nvidia-smi

alpha = "abcdefghijklmnopqrstuvwxyz"

class XGBoost:
    def __init__(self, num_classes=26):
        # Initialize 26 CatBoost classifiers, one for each label
        self.classifiers = [XGBClassifier(objective='binary:logistic' , tree_method='hist', device = 'cuda', learning_rate = 0.05) for _ in range(num_classes)]
        
    def fit(self, X, y):
        # X is the feature matrix, y is a binary matrix indicating label presence
        for i in range(len(self.classifiers)):
            # Train each classifier on the corresponding label
            dd = []
            self.classifiers[i].fit(X.drop(dd), y[alpha[i]].drop(dd), verbose=100)
    
    def predict(self, X):
        # Predict probabilities for each label
        predictions = np.zeros((len(X), len(self.classifiers)))
        for i, clf in enumerate(self.classifiers):
            predictions[:, i] = clf.predict(X)[:, 1]  # Probability of class '1'
        return predictions
    
    def save(self, filename):
        # Save the model to a pickle file
        with open(filename, 'wb') as file:
            pickle.dump(self, file)
    
    @classmethod
    def load(cls, filename):
        # Load the model from a pickle file
        with open(filename, 'rb') as file:
            model = pickle.load(file)
        return model

Sat Jan 20 12:00:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              25W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
model = XGBoost()

#train test split
from sklearn.model_selection import train_test_split
for i in range(0, 10):
    print(i)
    data = pd.read_parquet(file_paths[i])
    sub_dataset = np.array_split(data , 20)
    for sub_data in sub_dataset:
        X_train, X_test, y_train, y_test = train_test_split(sub_data[[str(x) for x in range(80)]], sub_data[[x for x in "abcdefghijklmnopqrstuvwxyz"]], test_size=0.01, random_state=42)
        model.fit(X_train,y_train)
    


0


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


1


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


2


...

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


8


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


9


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
model.save('/kaggle/working/multilabelxgboost_model.pkl')